In [1]:
import pandas as pd
import numpy as np
import pyodbc as pc
import sqlalchemy
import warnings

warnings.filterwarnings('ignore')

path_start = 'c:/tt/'
path_start_kntr = 'c:/tt/kntr/'

In [80]:
def get_dict_ru_en():
    s2 = """А а

A a

Л л

L l

Ц ц

Ts ts

Б б

B b

М м

M m

Ч ч

Ch ch

В в

V v

Н н

N n

Ш ш

Sh sh

Г г

G g

О о

O o

Щ щ

Shch shch

Д д

D d

П п

P p

Е е

E e

Р р

R r

Ы ы

Y y

Ж ж

Zh zh

С с

S s

З з

Z, z

Т т

T t

Э э

E e

И и

I i

У у

U u

Ю ю

Yu yu

Й й

Y y

Ф ф

F f

Я я

Ya ya

К к

K k

Х х

Kh kh

/ -

_ _"""
    ls1 = s2.split('\n\n')
    ls_ru = []
    cnt = len(ls1)
    for i in range(0, cnt, 2):
        ls_ru = ls_ru + ls1[i].split(' ')

    ls_en = []
    cnt = len(ls1)
    for i in range(1, cnt, 2):
        ls_en = ls_en + ls1[i].split(' ')

    ds = dict(zip(ls_ru, ls_en))
    return ds

ds_ru_en = get_dict_ru_en()

def ru_en(s1):
    s2 = ''
    for s_ru in str(s1):
        s_en = ds_ru_en.get(s_ru)
        if s_en:
            s2 += s_en
        else:
            s2 += s_ru
    return s2

def set_nameEn(path, typ):
    df1 = pd.read_excel(f'{path_start_kntr}{path}', sheet_name=typ)
    rezerv = 1
    ls1=[]
    for index, row in df1.iterrows():
        name = ru_en(row['name'])
        if row['name'] == 'резерв':
            name += str(rezerv)
            rezerv += 1
        ls1.append(name)
    df1['name_en'] = ls1
    return df1

def set_nameEns():
    df1 = set_nameEn('base/modbus_read_par.xlsx', 'sv')
    df2 = set_nameEn('base/modbus_read_par.xlsx', 'str')
    df3 = set_nameEn('base/modbus_read_par.xlsx', 'rs')
    with pd.ExcelWriter(f'{path_start_kntr}base/modbus_read_par.xlsx') as writer:  # doctest: +SKIP
        df1.to_excel(writer, sheet_name='sv', index=False)
        df2.to_excel(writer, sheet_name='str', index=False)
        df3.to_excel(writer, sheet_name='rs', index=False)

def set_reg_rd_wr(typ, start_reg_rd, start_reg_wr=-1):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    ls2 = df1['name_en'].tolist()
    ls_reg_rd = []
    ls_reg_wr = []
    ls_bit_wr = []
    reg_rd = start_reg_rd
    reg_wr = start_reg_wr
    bit = 0
    for sv in ls2:
        ls_reg_rd.append(reg_rd)
        reg_rd += 1
        if reg_wr != -1:
            ls_reg_wr.append(reg_wr)
            ls_bit_wr.append(bit)
            bit += 2
            if bit==16:
                bit = 0
                reg_wr += 1
    df1['reg_rd'] = ls_reg_rd
    if reg_wr != -1:
        df1['reg_wr'] = ls_reg_wr
        df1['bit_wr'] = ls_bit_wr
    return df1

def set_reg_rd_wr_rs(typ, start_reg_rd):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    ls2 = df1['name_en'].tolist()
    ls_reg_rd = []
    ls_bit_rd = []
    reg_rd = start_reg_rd

    bit = 0
    for sv in ls2:
        ls_reg_rd.append(reg_rd)
        ls_bit_rd.append(bit)
        bit += 1
        if bit==16:
            bit = 0
            reg_rd += 1
    df1['reg_rd'] = ls_reg_rd
    df1['bit_rd'] = ls_bit_rd
    return df1

def write_reg():
    df1 = set_reg_rd_wr('sv', 501, 401)
    df2 = set_reg_rd_wr('str', 701, 430)
    df3 = set_reg_rd_wr_rs('rs', 901)
    with pd.ExcelWriter(f'{path_start_kntr}base/modbus_read_par.xlsx') as writer:  # doctest: +SKIP
        df1.to_excel(writer, sheet_name='sv', index=False)
        df2.to_excel(writer, sheet_name='str', index=False)
        df3.to_excel(writer, sheet_name='rs', index=False)

def write_mdb_data():
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='sv')
    df1['gr_name']=''
    df1['gr_num']=0
    df1['gr_adr']=''
    df1['slave']=''
    slave=1
    ls1 = get_ls_dev_modbus('sv')
    # ls1 = [[6, 1, 0], ['M1', 'M3', 'M5', 'M7', 'M9', 'M11', 'M13', 'M15', 'M17', 'M19', 'M21', 'M23', 'M25', 'M27', 'M29', 'M33', 'M3
    for ls2, ls3 in ls1:
        gr = f'GR{ls2[0]}{ls2[1]}{ls2[2]}'
        adr = r"0.0.3{192.168.11." + f"{ls2[0]}" + r"}\\0." + f"{ls2[1]}.{ls2[2]}"
        i=1
        for sv in ls3:
            adr1 = f'{adr}.{slave}'
            df1.loc[df1['name_en']==sv, 'slave']= slave
            df1.loc[df1['name_en']==sv, 'gr_name']= gr
            df1.loc[df1['name_en']==sv, 'gr_num']= i
            df1.loc[df1['name_en']==sv, 'gr_adr']= adr1
            i+=1
            slave+=1

#     ------------------------------------------------------------------------

    df2 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='str')
    df2['gr_name']=''
    df2['gr_num']=0
    df2['gr_adr']=''
    df2['slave']=''
    slave=1
    ls1 = get_ls_dev_modbus('str')
    # ls1 = [[6, 1, 0], ['M1', 'M3', 'M5', 'M7', 'M9', 'M11', 'M13', 'M15', 'M17', 'M19', 'M21', 'M23', 'M25', 'M27', 'M29', 'M33', 'M3
    for ls2, ls3 in ls1:
        gr = f'GR{ls2[0]}{ls2[1]}{ls2[2]}'
        adr = r"0.0.3{192.168.11." + f"{ls2[0]}" + r"}\\0." + f"{ls2[1]}.{ls2[2]}"
        i=1
        for sv in ls3:
            adr1 = f'{adr}.{slave}'
            df2.loc[df2['name_en']==sv, 'slave']= slave
            df2.loc[df2['name_en']==sv, 'gr_name']= gr
            df2.loc[df2['name_en']==sv, 'gr_num']= i
            df2.loc[df2['name_en']==sv, 'gr_adr']= adr1
            i+=1
            slave+=1

    df3 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='rs')

    with pd.ExcelWriter(f'{path_start_kntr}base/modbus_read_par.xlsx') as writer:  # doctest: +SKIP
        df1.to_excel(writer, sheet_name='sv', index=False)
        df2.to_excel(writer, sheet_name='str', index=False)
        df3.to_excel(writer, sheet_name='rs', index=False)



def st_script_DDI(typ='rs', open_mod='a', ls_name_dop=[]):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    var_typ = 'RS'
    if typ == 'str':
        var_typ = 'STR'
    with open(f'{path_start_kntr}script_ddi.txt', open_mod, encoding='Windows-1251') as f:
        for index, row in df1.iterrows():
            rs_name = str(row['name_en'])
            if len(ls_name_dop) != 0:
                n1 = int(row["n"])
                i = 0
                for name_dop in ls_name_dop:
                    n1 += i
                    st1 = f'E_{name_dop}_{var_typ}_{rs_name}:=EIO2_d{row["d"]}_r0_s{row["s"]}_DDI{row["ddi"]}02K.DIS_CH_IN[{n1}].VALUE;\n'
                    f.write(st1)
                    i += 1
            else:
                st1 = f'E_{var_typ}_{rs_name}:=EIO2_d{row["d"]}_r0_s{row["s"]}_DDI{row["ddi"]}02K.DIS_CH_IN[{row["n"]}].VALUE;\n'
                f.write(st1)

def st_script_DDI_RS( open_mod='a'):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='rs')

    ls_gr = df1['reg_rd'].unique().tolist()

    with open(f'{path_start_kntr}script_ddi.txt', open_mod, encoding='Windows-1251') as f:

        f.write('\n(* Упаковка состояний рельсовых цепей в регистры-------------------------------------------------------------- *)\n')

        for gr_name in ls_gr:
            df2 = df1[df1['reg_rd']==gr_name]
            for index, row in df2.iterrows():
                ddi = f'EIO2_d{row["d"]}_r0_s{row["s"]}_DDI{row["ddi"]}02K.DIS_CH_IN[{row["n"]}].VALUE'
                s1=f'ARM_RS_REGRD{gr_name}.{row["bit_rd"]} := {ddi};\n'
                f.write(s1)

def st_script_Modbus_Read(typ='sv', open_mod='a'):
    dfstr1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    slave = 1
    var_typ = 'BKC2'
    typ_name = 'SV'
    if typ == 'str':
        var_typ = 'PCPT'
        typ_name = 'STR'
    with open(f'{path_start_kntr}script_modbus_read.txt', open_mod, encoding='Windows-1251') as f:
        for index, row in dfstr1.iterrows():
            name = str(row['name'])
            st1 = f"MDB_{var_typ}_{name}" + r" ('0.0.3{192.168.11." + f"{row['sh']}" \
                  + r"}\\0." + f"{row['pol']}.{row['canal']}.{slave}', ARM_{typ_name}_{name});\n"
            f.write(st1)
            slave += 1

def var_xml(ls, start_name_var, typeName, open_mod='a'):
    with open(f'{path_start_kntr}var.txt', open_mod, encoding='Windows-1251') as f:
        for st in ls:
            st1 = f'        <variables name="{start_name_var}_{ru_en(st)}" typeName="{typeName}"></variables>\n'
            f.write(st1)

def var_xml_ARM(ls, typ, reg_start, open_mod='a'):
    with open(f'{path_start_kntr}var.txt', "a", encoding='Windows-1251') as f:
        reg = reg_start
        ls_mw = []
        for st in ls:
            st1 = ru_en(st)
            s1 = f'''   <variables name="ARM_{typ}_{st1}" typeName="WORD" topologicalAddress="%MW{reg}">
            <attribute name="TimeStampSource" value="0"></attribute>
        </variables>
    '''
            ls_mw.append([st, reg, st1])
            reg += 1
            f.write(s1)
    return ls_mw

def write_var(typ, start_name_var, typeName, open_mod='a'):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    with open(f'{path_start_kntr}var.txt', open_mod, encoding='Windows-1251') as f:
        for index, row in df1.iterrows():
            name = row['name_en']
            s1 = f'        <variables name="{start_name_var}_{name}" typeName="{typeName}"></variables>\n'
            f.write(s1)

def write_var_ARM(typ, open_mod='a'):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    typ1 = 'SV'
    if typ=='str':
        typ1='STR'
    with open(f'{path_start_kntr}var.txt', open_mod, encoding='Windows-1251') as f:
        for index, row in df1.iterrows():
            s1 = f'''   <variables name="ARM_{typ1}_{row['name_en']}" typeName="WORD" topologicalAddress="%MW{row['reg_rd']}">
            <attribute name="TimeStampSource" value="0"></attribute>
        </variables>
    '''
            f.write(s1)


        ls_gr = df1['reg_wr'].unique().tolist()
        for reg in ls_gr:
            s2 = f'''   <variables name="ARM_{typ1}_REGWR{reg}" typeName="WORD" topologicalAddress="%MW{reg}">
            <attribute name="TimeStampSource" value="0"></attribute>
        </variables>
    '''
            f.write(s2)

def write_var_ARM_RS(open_mod='a'):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='rs')
    with open(f'{path_start_kntr}var.txt', open_mod, encoding='Windows-1251') as f:
        ls_gr = df1['reg_rd'].unique().tolist()
        for reg in ls_gr:
            s2 = f'''   <variables name="ARM_RS_REGRD{reg}" typeName="WORD" topologicalAddress="%MW{reg}">
            <attribute name="TimeStampSource" value="0"></attribute>
        </variables>
    '''
            f.write(s2)

def get_names_dev(typ='sv'):
    df = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    return df['name_en'].tolist()


def st_run_modbus(typ='sv', open_mod='a'):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    ls_gr = df1['gr_name'].unique().tolist()
    var_typ = 'BKC2'
    typ_name = 'SV'
    if typ == 'str':
        var_typ = 'PCPT'
        typ_name = 'STR'
    with open(f'{path_start_kntr}st_mdb_set_var.txt', open_mod, encoding='Windows-1251') as f:
        f.write('\n(* Запуск чтения, записи MODBUS-------------------------------------------------------------- *)\n')
        for gr_name in ls_gr:
            s1 = f'MDB_{var_typ}_{gr_name} (MDB_ADR_{typ_name}_{gr_name}, MDB_WR_{typ_name}_{gr_name}, MDB_RD_{typ_name}_{gr_name}, MDB_ERR_{typ_name}_{gr_name});\n'
            f.write(s1)


def get_ls_dev_modbus(typ='sv'):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    sh=0
    pol=0
    cnl=0
    ls1=[]
    ls2=[]
    ls3=[]
    sh1=pol1=cnl1=-1
    for index, row in df1.iterrows():

        sh = row['sh']
        pol = row['pol']
        cnl = row['canal']

        name = str(row['name_en'])

        s1 = f"{row['sh']},{row['pol']},{row['canal']}"
        if s1 in ls1:
            ls2.append(name)
        else:
            if sh1 != -1:
                ls3.append([[sh1, pol1, cnl1],ls2])
            ls1.append(s1)
            ls2 = [name]

        sh1 = row['sh']
        pol1 = row['pol']
        cnl1 = row['canal']

    ls3.append([[sh, pol, cnl],ls2])
    return ls3

def write_var_Modbus(typ='sv', open_mod='a'):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    ls_gr = df1['gr_name'].unique().tolist()
    typ1 = 'SV'
    cnt_arr = 18
    if typ=='str':
        typ1='STR'
        cnt_arr = 9
    with open(f'{path_start_kntr}var.txt', open_mod, encoding='Windows-1251') as f:
        for gr_name in ls_gr:
            name_var = f'{typ1}_{gr_name}'
            s1=f'		<variables name="MDB_ADR_{name_var}" typeName="ARRAY[1..{cnt_arr}] OF STRING"> </variables>\n'
            f.write(s1)

        #     df2 = df1[df1['gr_name']==gr_name]
        #     i=1
        #     for index, row in df2.iterrows():
        #         adr = row["gr_adr"]
        #         s1=f'''		     <instanceElementDesc name="[{i}]">
        #                 <value>'{adr}'</value>
        #             </instanceElementDesc>
        # '''
        #         f.write(s1)
        #         i+=1
        #     f.write('         </variables>\n')

        for gr_name in ls_gr:
                name_var = f'{typ1}_{gr_name}'
                s1=f'		<variables name="MDB_RD_{name_var}" typeName="ARRAY[1..{cnt_arr}] OF WORD"></variables>\n'
                s2=f'		<variables name="MDB_WR_{name_var}" typeName="ARRAY[1..{cnt_arr}] OF WORD"></variables>\n'
                s3=f'		<variables name="MDB_ERR_{name_var}" typeName="BOOL"></variables>\n'
                f.write(s1)
                f.write(s2)
                f.write(s3)

    with open(f'{path_start_kntr}var.txt', 'a', encoding='Windows-1251') as f:
        for gr_name in ls_gr:
            s1 = f'        <variables name="MDB_BKC2_{gr_name}" typeName="MODBUS_RW_BKC2"></variables>\n'
            if typ=='str':
                s1 = f'        <variables name="MDB_PCPT_{gr_name}" typeName="MODBUS_RW_PCPT"></variables>\n'
            f.write(s1)

def st_set_var_RD_Modbus(typ='sv', open_mod='a'):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    ls_gr = df1['gr_name'].unique().tolist()
    typ1 = 'SV'
    if typ=='str':
        typ1='STR'

    with open(f'{path_start_kntr}st_mdb_set_var.txt', open_mod, encoding='Windows-1251') as f:
        f.write('\n(* Заполнение данных чтения MODBUS-------------------------------------------------------------- *)\n')

        for gr_name in ls_gr:
            df2 = df1[df1['gr_name']==gr_name]
            for index, row in df2.iterrows():
                s1=f'MDB_RD_{typ1}_{gr_name}[{row["gr_num"]}] := ARM_{typ1}_{row["name_en"]};\n'
                f.write(s1)



def st_set_var_WR_Modbus(typ='sv', open_mod='a'):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    ls_gr = df1['gr_name'].unique().tolist()
    typ1 = 'SV'
    if typ=='str':
        typ1='STR'

    with open(f'{path_start_kntr}st_mdb_set_var.txt', open_mod, encoding='Windows-1251') as f:

        f.write('\n(* Заполнение данных записи MODBUS-------------------------------------------------------------- *)\n')

        for gr_name in ls_gr:
            df2 = df1[df1['gr_name']==gr_name]
            for index, row in df2.iterrows():
                s1=f'MDB_WR_{typ1}_{gr_name}[{row["gr_num"]}].0 := ARM_{typ1}_REGWR{row["reg_wr"]}.{row["bit_wr"]};\n'
                s2=f'MDB_WR_{typ1}_{gr_name}[{row["gr_num"]}].1 := ARM_{typ1}_REGWR{row["reg_wr"]}.{int(row["bit_wr"])+1};\n'
                f.write(s1)
                f.write(s2)

def st_set_var_RD_rs_Modbus(open_mod='a'):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='rs')
    ls_gr = df1['reg_rd'].unique().tolist()

    with open(f'{path_start_kntr}ARM_RS_REGRD.txt', open_mod, encoding='Windows-1251') as f:

        f.write('\n(* Упаковка состояний рельсовых цепей в регистры-------------------------------------------------------------- *)\n')

        for gr_name in ls_gr:
            df2 = df1[df1['reg_rd']==gr_name]
            for index, row in df2.iterrows():
                s1=f'ARM_RS_REGRD{gr_name}.{row["bit_rd"]} := E_RS_{row["name_en"]};\n'
                f.write(s1)

def st_set_adr_Modbus(typ='sv', open_mod='a'):
    df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name=typ)
    ls_gr = df1['gr_name'].unique().tolist()
    typ1 = 'SV'
    if typ=='str':
        typ1='STR'
    with open(f'{path_start_kntr}st_mdb_set_var.txt', open_mod, encoding='Windows-1251') as f:
        f.write('\n(* Заполнение данных адресса MODBUS-------------------------------------------------------------- *)\n')
        for gr_name in ls_gr:
            name_var = f'{typ1}_{gr_name}'
            df2 = df1[df1['gr_name']==gr_name]
            for index, row in df2.iterrows():
                gr_num = row["gr_num"]
                gr_adr = row["gr_adr"]
                s1=f"MDB_ADR_{name_var}[{gr_num}] := '{gr_adr}';\n"
                f.write(s1)

def kntr_write_data1():
    # Переменные ----------------------------------------------------
    write_var('rs', 'E_RS', 'EBOOL', 'w')
    write_var('str', 'E_SD_STR', 'EBOOL')
    write_var('str', 'E_OF_STR', 'EBOOL')
    write_var_ARM('sv')
    write_var_ARM('str')
    write_var_ARM_RS()
    write_var_Modbus('sv')
    write_var_Modbus('str')

    # EIO (st script)----------------------------------------------------
    # st_script_DDI('rs', 'w')
    st_script_DDI_RS('w')
    st_script_DDI('str', 'a', ['SD', 'OF'])

    # MODBUS RUN (st script)----------------------------------------------------
    st_run_modbus('sv', 'w')
    st_run_modbus('str', 'a')

    st_set_var_WR_Modbus('sv', 'w')
    st_set_var_WR_Modbus('str')
    st_set_adr_Modbus('sv')
    st_set_adr_Modbus('str')
    st_run_modbus('sv')
    st_run_modbus('str')
    st_set_var_RD_Modbus('sv')
    st_set_var_RD_Modbus('str')

    st_set_var_RD_rs_Modbus('w')


In [70]:
# set_nameEns()
# write_reg()
# write_mdb_data()

In [81]:
kntr_write_data1()

In [59]:
df1 = pd.read_excel(f'{path_start_kntr}base/modbus_read_par.xlsx', sheet_name='str')
ls_gr = df1['reg_wr'].unique().tolist()